In [1]:
from bokeh.plotting import figure, output_server, show, cursession, push
from bokeh.models import LinearAxis, Range1d, ColumnDataSource
from bokeh.models.widgets import TextInput,VBox
from bokeh.io import gridplot, vplot, hplot
from bokeh.document import Document
from bokeh.session import Session
from datetime import datetime
import numpy as np
import pandas as pd
import os
from multiprocessing import Pool, Process, Queue
import subprocess
from time import sleep
import sys

In [2]:
document = Document()
session = Session()
session.use_doc('Stock Price')
session.load_document(document)

Using saved session configuration for http://localhost:5006/
To override, pass 'load_from_config=False' to Session


In [3]:
def load_data(files,q):
    '''
    files: files to load. If not available, download first.
    q: queue to put data.
    '''
    for f in files:
        if not os.path.exists(f):
            print("Downloading {}".format(f))
            sys.stdout.flush()
            s,t = f.split('/')
            if not os.path.exists(s):
                os.makedirs(s)
            subprocess.call(['scp', '-q', '-r', 'osg:/home/bill10/stash/public/Stock/{}.KP/{}'.format(t,s),'{}/{}'.format(s,t)])
        if os.path.exists(f):
            df=pd.read_csv(f, header=None,
                usecols=[1,2,6,8],
                parse_dates={'Time':[1,2]})
            df.columns=['Time', 'Price', 'Size']
            if df.size:
                q.put(df)

def update_data(ds,s,q):
    '''
    ds: data source to update.
    s: session to update.
    q: queue to get data.
    '''
    print("Waiting to update!")
    while True:
        df=q.get()
        if df.size:
            ds.data=ds.to_df().append(df).to_dict('list')
        else:
            ds.data={'Time':[], 'Price':[], 'Size':[]}
        s.store_objects(ds)

def input_change(obj, attrname, old, new):
    q.put(pd.DataFrame({'Time':[], 'Price':[], 'Size':[]}))
    print("Pulling {}...".format(new))
    files=["e{}/201004{:0>2}".format(new,i) for i in range(1,2)]
    chunks=10
    if len(files)<chunks:
        chunksize=1
    else:
        chunksize=int(np.ceil(len(files)*1.0/chunks))
    jobs=[Process(target=load_data, args=(files[i:i+chunksize],q), name='load_data') for i in xrange(0, len(files), chunksize)]
    print("Starting jobs!!!")
    for p in jobs: 
        p.start()
    for p in jobs:
        p.join()

text_input = TextInput(value="", title="Symbol:")
text_input.on_change('value',input_change)
data=ColumnDataSource({'Time':[], 'Price':[], 'Size':[]})
p = figure(x_axis_type = "datetime",plot_width=900,plot_height=300)
p.line('Time', 'Price', source=data)
layout = VBox([text_input,p]);
document.add(layout)
session.store_document(document)
q=Queue()
p=Process(target=update_data, args=(data,session,q), name='stockapp_update')
p.start()
session.poll_document(document,1);

Pulling GOOG...
Starting jobs!!!
Waiting to update!
Pulling AAPL...
Starting jobs!!!
Connection to bokeh-server was terminated


/home/ubuntu/anaconda/lib/python2.7/site-packages/bokeh/session.py:318: UserWarning: You need to start the bokeh-server to see this example.
  warnings.warn("You need to start the bokeh-server to see this example.")


In [13]:
%%file stockapp.py
import numpy as np
from bokeh.plotting import figure
from bokeh.models import Plot, ColumnDataSource, Range1d
from bokeh.properties import Instance
from bokeh.server.app import bokeh_app
from bokeh.server.utils.plugins import object_page
from bokeh.models.widgets import HBox, Slider, TextInput, VBoxForm, VBox
from multiprocessing import Pool,Process,Queue
import os
import sys
import subprocess
import pandas as pd


class StockApp(VBox):
    extra_generated_classes = [["StockApp", "StockApp", "VBox"]]
    text = Instance(TextInput)
    plot = Instance(Plot)
    source = Instance(ColumnDataSource)

    @classmethod
    def create(cls):
        """One-time creation of app's objects.
        This function is called once, and is responsible for
        creating all objects (plots, datasources, etc)"""
        obj = cls()
        obj.source = ColumnDataSource({'Time':[], 'Price':[], 'Size':[]})
        obj.text = TextInput(value="", title="Symbol:")
        # Generate a figure container
        plot = figure(x_axis_type = "datetime",
                      plot_width=900,plot_height=300)
                      #tools="pan,box_zoom,reset,resize,crosshair")
        # Plot the line by the x,y values in the source property
        plot.line('Time', 'Price', source=obj.source)
        obj.plot = plot
        obj.children.append(obj.text)
        obj.children.append(obj.plot)
        return obj

    def setup_events(self):
        """Attaches the on_change event to the value property of the widget.
        The callback is set to the input_change method of this app."""
        super(StockApp, self).setup_events()
        if not self.text:
            return
        # Text box event registration
        self.text.on_change('value', self, 'input_change')

    def input_change(self, obj, attrname, old, new):
        """Executes whenever the input form changes.
        It is responsible for updating the plot, or anything else you want.
        Args:
            obj : the object that changed
            attrname : the attr that changed
            old : old value of attr
            new : new value of attr
        """
        print("Pulling {}...".format(new))
        def load_data(files,q):
            for f in files:
                if not os.path.exists(f):
                    print("Downloading {}".format(f))
                    sys.stdout.flush()
                    _,s,t = f.split('/')
                    if not os.path.exists("Data/"+s):
                        os.makedirs("Data/"+s)
                    subprocess.call(['scp', '-q', '-r', 'osg:/home/bill10/stash/public/Stock/{}.KP/e{}'.format(t,s),'Data/{}/{}'.format(s,t)])
                if not os.path.exists(f):
                    q.put(pd.DataFrame(columns=['Time', 'Price', 'Size']))
                else:
                    df=pd.read_csv(f, header=None,
                        usecols=[1,2,6,8],
                        parse_dates={'Time':[1,2]})
                    df.columns=['Time', 'Price', 'Size']
                    q.put(df)
        q=Queue()
        files=["Data/{}/201004{:0>2}".format(new,i) for i in range(1,2)]
        chunks=10
        if len(files)<chunks:
            chunksize=1
        else:
            chunksize=int(np.ceil(len(files)*1.0/chunks))
        jobs=[Process(target=load_data, args=(files[i:i+chunksize],q), name='stockapp') for i in xrange(0, len(files), chunksize)]
        for p in jobs: 
            p.start()
        res=0
        df=pd.DataFrame({'Time':[], 'Price':[], 'Size':[]})
        while res<len(files):
            df=df.append(q.get())
            res+=1
        for p in jobs:
            p.join()
        self.source.data=df.sort('Time').to_dict('list')
        #self.plot.y_range.start=df['Price'].min()
        #self.plot.y_range.end=df['Price'].max()
        self.plot.line('Time','Price',source=self.source)

# The following code adds a "/bokeh/sliders/" url to the bokeh-server. This
# URL will render this sine wave sliders app. If you don't want to serve this
# applet from a Bokeh server (for instance if you are embedding in a separate
# Flask application), then just remove this block of code.
@bokeh_app.route("/stocks/")
@object_page("stocks")
def make_sliders():
    app = StockApp.create()
    return app


Writing stockapp.py


In [22]:
p=figure()

In [23]:
p.line([1,2],[1,2])

In [35]:
p.y_range.update

<bound method DataRange1d.update of <bokeh.models.ranges.DataRange1d object at 0x7fc3559ba9d0>>

In [24]:
p.properties()

{'above',
 'background_fill',
 'below',
 'border_fill',
 'disabled',
 'extra_x_ranges',
 'extra_y_ranges',
 'h_symmetry',
 'left',
 'lod_factor',
 'lod_interval',
 'lod_threshold',
 'lod_timeout',
 'logo',
 'min_border',
 'min_border_bottom',
 'min_border_left',
 'min_border_right',
 'min_border_top',
 'name',
 'outline_line_alpha',
 'outline_line_cap',
 'outline_line_color',
 'outline_line_dash',
 'outline_line_dash_offset',
 'outline_line_join',
 'outline_line_width',
 'plot_height',
 'plot_width',
 'renderers',
 'responsive',
 'right',
 'session',
 'tags',
 'title',
 'title_text_align',
 'title_text_alpha',
 'title_text_baseline',
 'title_text_color',
 'title_text_font',
 'title_text_font_size',
 'title_text_font_style',
 'tool_events',
 'toolbar_location',
 'tools',
 'v_symmetry',
 'webgl',
 'x_mapper_type',
 'x_range',
 'y_mapper_type',
 'y_range'}